In [3]:
#https://github.com/amazon-science/efficient-longdoc-classification
from functools import partial
import nltk
import pickle as pk
import torch
from context_enforcement.models.context_enforcer import compute_context_boundary
from context_enforcement.trainers.train_bart import model_init
from context_enforcement.data.common import create_text_tokenizer, SmartCollator
from context_enforcement.trainers.common import get_dataset_specified_tasks
from pytorch_lightning import seed_everything

import sys
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
seed_everything(1376)

Global seed set to 1376
[nltk_data] Downloading package punkt to /home/nlplab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Global seed set to 1376


1376

In [2]:
z=torch.ones((1,2),dtype=torch.long,device='cuda')
z.device

device(type='cuda', index=0)

In [13]:
configs

Namespace(is_baseline=False, model_base='facebook/bart-base', output_dir='trained_models_sum_boundary/', task_type='xsum', run_id='bart_base_model_context_enforcer_250', eval_steps=1000, learning_rate=5e-05, max_seq_len=800, evaluation_strategy='epoch', save_strategy='epoch', seed=10, lr_scheduler_type='linear', weight_decay=0.0, warmup_ratio=0.25, num_train_epochs=10, save_total_limit=1, per_device_train_batch_size=12, per_device_eval_batch_size=12, gradient_accumulation_steps=4, fp16=True, verbose=False, context_max_len=250)

In [5]:
configs= pk.load(open("trained-models/bart-base-context-enhance-form1/train_args.ap",'rb'))
context_max_len=configs.context_max_len
context_max_len_list = [context_max_len]#,300,450]
context_sampling_bounds=(0.1, 0.45)

In [3]:
configs.model_base

'facebook/bart-base'

In [6]:
tokenizer = create_text_tokenizer(configs.model_base)

task_dataset_gen = get_dataset_specified_tasks(configs.task_type)

train_dataset = None
eval_dataset = None
test_dataset = None
if task_dataset_gen is not None:
    raw_dataset = task_dataset_gen(tokenizer=tokenizer, )
    train_dataset = raw_dataset['train']
    eval_dataset = raw_dataset['validation']
    test_dataset = raw_dataset['test']

model_builder = model_init(
        vocab_size=len(train_dataset.tokenizer),
        model_base=configs.model_base,
        context_max_len = context_max_len,
        context_sampling_bounds = context_sampling_bounds,
        context_max_len_list= context_max_len_list,#is_baseline=True
    )

Found cached dataset xsum (/home/nlplab/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71)
100%|██████████| 3/3 [00:00<00:00, 791.03it/s]


In [4]:
generator = model_builder()
train_model_path = "trained-models/bart-base-context-enforcer/checkpoint-29757/pytorch_model.bin"
state_dict = torch.load(train_model_path)
generator.load_state_dict(state_dict)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

Some weights of BartForContextualRecovery were not initialized from the model checkpoint at facebook/bart-base and are newly initialized: ['encoder.layers.5.context_enforcer._context_mha.out_proj.weight', 'encoder.layers.1.context_enforcer_layer_norm.weight', 'encoder.layers.5.context_enforcer._right_mha.out_proj.weight', 'encoder.layers.1.context_enforcer._wlc.bias', 'encoder.layers.3.context_enforcer._context_mha.out_proj.bias', 'encoder.layers.3.context_enforcer._right_mha.in_proj_weight', 'encoder.layers.1.context_enforcer._wfc.bias', 'encoder.layers.1.context_enforcer._wrc.bias', 'encoder.layers.3.context_enforcer._wrc.weight', 'encoder.layers.3.context_enforcer._right_mha.out_proj.weight', 'encoder.layers.1.context_enforcer._wlc.weight', 'encoder.layers.3.context_enforcer_layer_norm.bias', 'encoder.layers.5.context_enforcer._right_mha.in_proj_bias', 'encoder.layers.3.context_enforcer._context_mha.in_proj_weight', 'encoder.layers.5.context_enforcer._wrc.weight', 'encoder.layers.1.

In [5]:
te1= test_dataset[690]
b_input_ids = te1.input_ids.view(1, -1).to(device)
b_input_mask = te1.attention_mask.view(1, -1).to(device)

In [5]:
example= {'document':"""
          Gently exfoliating encourages skin cell renewal, therefore encouraging younger looking skin.
          Choose something that is aimed at older skin as they are more gentle and less likely to dry out or damage the skin.
          Exfoliation is also good to do as it leaves your skin feeling softer and looking brighter.
          For men, exfoliating before shaving your face can make the shaving process easier as it exposes more of the hair.
           """}
te1=test_dataset._process_data(example)
b_input_ids = te1.input_ids.view(1, -1).to(device)
b_input_mask = te1.attention_mask.view(1, -1).to(device)

In [6]:
#seed_everything(1376)
boundary_sample =  (0.15, 0.35)
seq_len = te1.input_ids[:1024].shape[0]

boundary_start = int(0.4*seq_len) 
context_boundary = compute_context_boundary(seq_len,
                                            context_sampling_bounds=boundary_sample,
                                            context_max_len=boundary_start)

context_boundary,seq_len,context_max_len

((57, 153), 241, 310)

In [7]:
generator.eval()
with torch.no_grad():
    bb=generator.generate(input_ids=b_input_ids[:,:1024],
                attention_mask=b_input_mask[:,:1024],
                context_boundary=context_boundary,
                eos_token_id=test_dataset.tokenizer.eos_token_id,
        max_length=189,
        early_stopping=True,
        use_cache=True,
        num_beams=10,
        )
test_dataset.tokenizer.batch_decode(bb,clean_up_tokenization_spaces=True,skip_special_tokens=True)

['The Democratic Republic of Congo singer Koffi Olomide has been arrested and taken into police custody in the capital, Kinshasa.']

In [ ]:
generator.eval()
with torch.no_grad():
    bb=generator.generate(input_ids=b_input_ids[:,:1024],
                attention_mask=b_input_mask[:,:1024],
                context_boundary=context_boundary,
                eos_token_id=test_dataset.tokenizer.eos_token_id,
        max_length=189,
        early_stopping=True,
        use_cache=True,
        num_beams=8,
        )
test_dataset.tokenizer.batch_decode(bb,clean_up_tokenization_spaces=True,skip_special_tokens=True)

["Uganda's popular rumba singer Oscar Pistorius has been taken into police custody in the Democratic Republic of Congo, his lawyer says."]

In [10]:
test_dataset.tokenizer.decode(te1.labels,clean_up_tokenization_spaces=True,skip_special_tokens=True)

'Musician Koffi Olomide has been taken into custody in the Democratic Republic of Congo, days after he was deported from Kenya for allegedly kicking one of his dancers at an airport in Nairobi.'

In [5]:
te1.input_ids.shape

NameError: name 'te1' is not defined

In [13]:
print(test_dataset.tokenizer.decode(te1.input_ids,clean_up_tokenization_spaces=True,skip_special_tokens=True))

A judge is considering whether to charge the star over the incident, which was filmed and went viral.
He will spend the night in detention and investigations will continue on Wednesday, a BBC reporter says.
Olomide has denied assault, though he has since apologised for his behaviour.
The Kenyan authorities deported the 60-year-old rumba singer and three of his dancers on Saturday to DR Congo's capital, Kinshasa, following a public outcry over the incident.
The BBC's Poly Muzalia in Kinshasa says police officers arrived at the singer's home early on Tuesday morning and took him into custody.
He was handcuffed, put into a police car and then taken to a court where a judge is considering whether he should be put on trial.
His lawyer, Landry Tanganyi, told the BBC that Olomide, one of Africa's most popular musicians, should not be detained overnight as he was not a flight risk.
However, Olomide left the courthouse under police escort and will spend the night at the police station, our repo

In [5]:
generator.model.context_max_len= 800
generator.model.encoder.context_max_len= 800

In [5]:
from torch.utils.data import DataLoader,SequentialSampler
import tqdm
from context_enforcement.data.common import write_to_file
import evaluate
metrics = evaluate.combine(['bleu','meteor',"rouge"])
def generate():
    # context_sampling_bounds = (0.05,0.25)
    test_data_loader = DataLoader(test_dataset,batch_size=12,
                                sampler= SequentialSampler(test_dataset),
                                collate_fn= SmartCollator(
                pad_token_id=train_dataset.tokenizer.pad_token_id,
                max_len=configs.max_seq_len,
                context_max_len=context_max_len,
                context_sampling_bounds=context_sampling_bounds,
            
            ))

    output_summaries =[]
    for batch in tqdm.tqdm(test_data_loader):
        b_input_ids = batch['input_ids'].to(device)
        b_input_mask = batch['attention_mask'].to(device)
        
        seq_len = b_input_ids.shape[1]

        context_boundary = compute_context_boundary(seq_len,
                                                    context_max_len=110)
        
        boundary_mask =  batch.get("boundary",)
        with torch.no_grad():
            bb=generator.generate(input_ids=b_input_ids,
                    attention_mask=b_input_mask,
                    context_boundary=boundary_mask,
                    num_beams=10,
                    do_sample=False,
                    num_return_sequences=1,
                    max_length=160)
        sentences = test_dataset.tokenizer.batch_decode(bb,clean_up_tokenization_spaces=True,skip_special_tokens=True)
        output_summaries+=sentences
    return output_summaries

[nltk_data] Downloading package wordnet to /home/nlplab/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/nlplab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/nlplab/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [6]:
targets = [tokenizer.decode(c.labels,clean_up_tokenization_spaces=True,skip_special_tokens=True) for c in test_dataset]

Token indices sequence length is longer than the specified maximum sequence length for this model (1327 > 1024). Running this sequence through the model will result in indexing errors


In [7]:
output_folder = 'outputs/bart-context-enforcer/best-29757/'
os.makedirs(output_folder,exist_ok=True)

In [8]:
context_lens = [70,100,200,300,400,500,600,700]
outputs = {}
results = {}
for cl in context_lens:
    print(f'Generating for the context length: {cl}')
    generator.model.context_max_len= cl
    generator.model.encoder.context_max_len= cl
    rbase_output = generate()
    outputs[cl] = rbase_output
    
    write_to_file(rbase_output[:len(test_dataset)], 
              f"{output_folder}/generated_with_contextlen_{cl}")
    
    scores = metrics.compute(predictions=rbase_output,references=targets)
    print(scores)
    
    results[cl]= scores

Generating for the context length: 70


100%|██████████| 945/945 [33:23<00:00,  2.12s/it] 


Done
{'bleu': 0.09289873176942315, 'precisions': [0.38470217484340347, 0.13430427331778738, 0.06649126086102206, 0.036194096034460915], 'brevity_penalty': 0.8797425367801427, 'length_ratio': 0.8864258179226413, 'translation_length': 231327, 'reference_length': 260966, 'meteor': 0.3019540336536042, 'rouge1': 0.3519294790864753, 'rouge2': 0.14156512810311236, 'rougeL': 0.2847612821341625, 'rougeLsum': 0.28466492829881795}
Generating for the context length: 100


100%|██████████| 945/945 [34:49<00:00,  2.21s/it]


Done
{'bleu': 0.1008520817476466, 'precisions': [0.4006564197590214, 0.14634707080553422, 0.07438309184133807, 0.04160870569514668], 'brevity_penalty': 0.8689228617701171, 'length_ratio': 0.876807706751071, 'translation_length': 228817, 'reference_length': 260966, 'meteor': 0.31552595580700815, 'rouge1': 0.36666610414014633, 'rouge2': 0.15333178255601693, 'rougeL': 0.2975021893222358, 'rougeLsum': 0.29737280327145404}
Generating for the context length: 200


...

100%|██████████| 945/945 [55:10<00:00,  3.50s/it]  


Done
{'bleu': 0.116453744376084, 'precisions': [0.43155483456930976, 0.17102088711570054, 0.0911330781891372, 0.052489205912323644], 'brevity_penalty': 0.8495631397674898, 'length_ratio': 0.8598208195703655, 'translation_length': 224384, 'reference_length': 260966, 'meteor': 0.3415006733023984, 'rouge1': 0.393336347460204, 'rouge2': 0.1761138745237874, 'rougeL': 0.32254237071368097, 'rougeLsum': 0.32249024830254003}
Generating for the context length: 500


100%|██████████| 945/945 [1:02:19<00:00,  3.96s/it]


Done
{'bleu': 0.11730734115252861, 'precisions': [0.4331752624229906, 0.17241233490111688, 0.09208260336587375, 0.05310292965360567], 'brevity_penalty': 0.8485807557348521, 'length_ratio': 0.8589663021236483, 'translation_length': 224161, 'reference_length': 260966, 'meteor': 0.34285716032154273, 'rouge1': 0.39438413600795896, 'rouge2': 0.17695651896552428, 'rougeL': 0.3235800092344216, 'rougeLsum': 0.3235640301975921}
Generating for the context length: 600


 64%|██████▍   | 609/945 [42:10<22:06,  3.95s/it]  

Generating for the context length: 70


100%|██████████| 945/945 [34:10<00:00,  2.17s/it] 


Done
{'bleu': 0.09250959688881132, 'precisions': [0.3801851305028822, 0.13212706722617265, 0.06531124854861259, 0.03561841419928683], 'brevity_penalty': 0.889763152364391, 'length_ratio': 0.8954154947387782, 'translation_length': 233673, 'reference_length': 260966, 'meteor': 0.3011444585413878, 'rouge1': 0.35035226251894036, 'rouge2': 0.1405239093157331, 'rougeL': 0.28208045988279706, 'rougeLsum': 0.2819424162036307}
Generating for the context length: 100


100%|██████████| 945/945 [35:11<00:00,  2.23s/it]


Done
{'bleu': 0.10272110045943982, 'precisions': [0.39918809201623817, 0.14667530537533074, 0.07490280736276017, 0.041793742804629275], 'brevity_penalty': 0.8828230796933534, 'length_ratio': 0.889180966102864, 'translation_length': 232046, 'reference_length': 260966, 'meteor': 0.31877442501002473, 'rouge1': 0.36800137226389906, 'rouge2': 0.15428097005955713, 'rougeL': 0.297750798313312, 'rougeLsum': 0.2976870208349942}
Generating for the context length: 200


...

Done
{'bleu': 0.11776022518819838, 'precisions': [0.43095652941046736, 0.17105367017843628, 0.09105701416899876, 0.05243820801959492], 'brevity_penalty': 0.8597394398761952, 'length_ratio': 0.8687146984664669, 'translation_length': 226705, 'reference_length': 260966, 'meteor': 0.3434435377556876, 'rouge1': 0.3951613255660041, 'rouge2': 0.1764529644897777, 'rougeL': 0.3230888548815952, 'rougeLsum': 0.32305850465656594}
Generating for the context length: 600


100%|██████████| 945/945 [1:05:10<00:00,  4.14s/it]


Done
{'bleu': 0.11722327853021175, 'precisions': [0.42985336056050416, 0.17019778590949095, 0.0903420090342009, 0.0517435902743994], 'brevity_penalty': 0.8620029054959953, 'length_ratio': 0.8707034632864051, 'translation_length': 227224, 'reference_length': 260966, 'meteor': 0.3432064956624229, 'rouge1': 0.39475088064300984, 'rouge2': 0.1760622468788845, 'rougeL': 0.32261314254736095, 'rougeLsum': 0.32252878573133154}
Generating for the context length: 700


100%|██████████| 945/945 [1:07:32<00:00,  4.29s/it]


Done
{'bleu': 0.11663778812401596, 'precisions': [0.4286631599317418, 0.16933132134161583, 0.08979795216507738, 0.05127475823550706], 'brevity_penalty': 0.862647557793925, 'length_ratio': 0.8712705869730156, 'translation_length': 227372, 'reference_length': 260966, 'meteor': 0.3423776366003322, 'rouge1': 0.39386609975080156, 'rouge2': 0.17522644619408478, 'rougeL': 0.32161319608822175, 'rougeLsum': 0.3215174418373331}


Generating for the context length: 70


100%|██████████| 945/945 [34:06<00:00,  2.17s/it] 


Done
{'bleu': 0.09790852957555751, 'precisions': [0.38666570447377246, 0.13750507271259682, 0.06935487658814965, 0.038353541166431845], 'brevity_penalty': 0.8978138834008965, 'length_ratio': 0.9026961366614809, 'translation_length': 235573, 'reference_length': 260966, 'meteor': 0.311213913533376, 'rouge1': 0.3583979171513669, 'rouge2': 0.1468928820952023, 'rougeL': 0.2895044686273106, 'rougeLsum': 0.2894119411256414}
Generating for the context length: 100


100%|██████████| 945/945 [35:20<00:00,  2.24s/it]


Done
{'bleu': 0.10664829981705916, 'precisions': [0.4033057426793464, 0.15041381792011516, 0.07766392082887016, 0.04382625768837778], 'brevity_penalty': 0.8896823525297948, 'length_ratio': 0.8953426883195512, 'translation_length': 233654, 'reference_length': 260966, 'meteor': 0.3260664197075103, 'rouge1': 0.37452467345345963, 'rouge2': 0.15935492013006633, 'rougeL': 0.3035347302941621, 'rougeLsum': 0.3034528449599872}
Generating for the context length: 200


100%|██████████| 945/945 [41:36<00:00,  2.64s/it] 


Done
{'bleu': 0.11750724691951889, 'precisions': [0.42545967042497834, 0.16779619275218227, 0.08899063155262298, 0.05081943865146136], 'brevity_penalty': 0.876619218597508, 'length_ratio': 0.8836400144080072, 'translation_length': 230600, 'reference_length': 260966, 'meteor': 0.3450049269432372, 'rouge1': 0.39363944073565926, 'rouge2': 0.1750110804618783, 'rougeL': 0.3216178154973085, 'rougeLsum': 0.32146277399903833}
Generating for the context length: 300


 51%|█████     | 484/945 [24:47<21:39,  2.82s/it] 

Generating for the context length: 70


100%|██████████| 945/945 [35:23<00:00,  2.25s/it] 


Done
{'bleu': 0.09807148092036545, 'precisions': [0.3992554758691261, 0.1441677345070292, 0.07291697295813854, 0.04032793690327937], 'brevity_penalty': 0.8598136500896202, 'length_ratio': 0.868779841052091, 'translation_length': 226722, 'reference_length': 260966, 'meteor': 0.3118438243077101, 'rouge1': 0.36306232062789434, 'rouge2': 0.14993312151411836, 'rougeL': 0.29462414304413453, 'rougeLsum': 0.2946266954059569}
Generating for the context length: 100


100%|██████████| 945/945 [34:34<00:00,  2.20s/it] 


Done
{'bleu': 0.10493294308065042, 'precisions': [0.4136273703399062, 0.1547843675233284, 0.07979846425410513, 0.04505717263884597], 'brevity_penalty': 0.8518990716995946, 'length_ratio': 0.8618555673919208, 'translation_length': 224915, 'reference_length': 260966, 'meteor': 0.3234253931411736, 'rouge1': 0.3753577716427966, 'rouge2': 0.16003048841535605, 'rougeL': 0.3057567993299293, 'rougeLsum': 0.30564745400788074}
Generating for the context length: 200


...

100%|██████████| 945/945 [48:02<00:00,  3.05s/it]  


Done
{'bleu': 0.11608114937730499, 'precisions': [0.4337649156371116, 0.17209170274682375, 0.09172954135229323, 0.052759624099604066], 'brevity_penalty': 0.8419869929414174, 'length_ratio': 0.8532490822559261, 'translation_length': 222669, 'reference_length': 260966, 'meteor': 0.3411440721656123, 'rouge1': 0.3934062110447026, 'rouge2': 0.17612771096375035, 'rougeL': 0.3229750760710789, 'rougeLsum': 0.3228282209872626}
Generating for the context length: 400


100%|██████████| 945/945 [54:39<00:00,  3.47s/it]  


Done
{'bleu': 0.1167485877592128, 'precisions': [0.43476927517408087, 0.17334435984448818, 0.09292804468824724, 0.05383148497341984], 'brevity_penalty': 0.8378450454460028, 'length_ratio': 0.8496739038801989, 'translation_length': 221736, 'reference_length': 260966, 'meteor': 0.3412216699978226, 'rouge1': 0.3936639053450192, 'rouge2': 0.17737029734817564, 'rougeL': 0.3233846297737608, 'rougeLsum': 0.32327174322026553}
Generating for the context length: 500


 73%|███████▎  | 687/945 [45:14<18:04,  4.20s/it]  

Generating for the context length: 200


100%|██████████| 945/945 [08:19<00:00,  1.89it/s]


Done
{'bleu': 0.12346216754890589, 'precisions': [0.4375689199934979, 0.1776036248468456, 0.09612053612814771, 0.05588868746029532], 'brevity_penalty': 0.863722634039824, 'length_ratio': 0.8722170704229669, 'translation_length': 227619, 'reference_length': 260966, 'meteor': 0.35359416748473077, 'rouge1': 0.40248422610308543, 'rouge2': 0.18328221126363148, 'rougeL': 0.33009269996078755, 'rougeLsum': 0.3300614517511684}
Generating for the context length: 720


100%|██████████| 945/945 [10:18<00:00,  1.53it/s]


Done
{'bleu': 0.1257888249451646, 'precisions': [0.4377841740444545, 0.1791020085562382, 0.09760767869694839, 0.05722976542823581], 'brevity_penalty': 0.8695117278704764, 'length_ratio': 0.8773288474360644, 'translation_length': 228953, 'reference_length': 260966, 'meteor': 0.3565514119827842, 'rouge1': 0.4047119203068985, 'rouge2': 0.185856595769887, 'rougeL': 0.332006465804801, 'rougeLsum': 0.33195165056651943}
Generating for the context length: 650


...

Done
{'bleu': 0.1256403068872618, 'precisions': [0.4381767948566864, 0.17878675170700573, 0.09728983506502943, 0.05701369989390655], 'brevity_penalty': 0.8702041264108997, 'length_ratio': 0.8779419541242921, 'translation_length': 229113, 'reference_length': 260966, 'meteor': 0.3563541597980937, 'rouge1': 0.4050806838364527, 'rouge2': 0.18561721029291084, 'rougeL': 0.3319262324656158, 'rougeLsum': 0.33192334002381724}
Generating for the context length: 350


100%|██████████| 945/945 [09:11<00:00,  1.71it/s]


Done
{'bleu': 0.1257626122759007, 'precisions': [0.4382902891885163, 0.17890146449955657, 0.09765710213638545, 0.05715340935321546], 'brevity_penalty': 0.8695030702543955, 'length_ratio': 0.8773211836024616, 'translation_length': 228951, 'reference_length': 260966, 'meteor': 0.3562012415308598, 'rouge1': 0.4047128146167146, 'rouge2': 0.18550529759109602, 'rougeL': 0.33186920040252776, 'rougeLsum': 0.3318646742987025}
Generating for the context length: 100


100%|██████████| 945/945 [07:49<00:00,  2.01it/s]


Done
{'bleu': 0.11964712510107044, 'precisions': [0.4326436377229051, 0.17240038692776577, 0.0924841374122612, 0.05362138236647655], 'brevity_penalty': 0.8627477081454109, 'length_ratio': 0.8713587210594483, 'translation_length': 227395, 'reference_length': 260966, 'meteor': 0.3474841689817138, 'rouge1': 0.39724258276498764, 'rouge2': 0.17816489836175736, 'rougeL': 0.32460150343422817, 'rougeLsum': 0.32467930362884756}


In [11]:
base_output = generate()

100%|██████████| 945/945 [08:28<00:00,  1.86it/s]


In [17]:
generator.model.context_max_len= 800
generator.model.encoder.context_max_len= 800
base_output8 = generate()

100%|██████████| 945/945 [10:04<00:00,  1.56it/s]


In [18]:
generator.model.context_max_len= 600
generator.model.encoder.context_max_len= 600
base_output6 = generate()

100%|██████████| 945/945 [10:02<00:00,  1.57it/s]


In [19]:
generator.model.context_max_len= 450
generator.model.encoder.context_max_len= 450
base_output45 = generate()

100%|██████████| 945/945 [09:30<00:00,  1.66it/s]


In [10]:
#os.makedirs("outputs/multi_context/")

In [25]:
#context_enforcement/data/common.py

write_to_file(base_output8[:len(test_dataset)], 
              "outputs/multi_context/best_base_final_800")

Done


In [14]:
targets[690],base_output[690]#,base_output_8[690],base_output_45[690]#output_summaries2[690],output_summaries[690],output_summaries6[690],output_summaries8[690]

('Musician Koffi Olomide has been taken into custody in the Democratic Republic of Congo, days after he was deported from Kenya for allegedly kicking one of his dancers at an airport in Nairobi.',
 "Uganda's Koffi Olomide has been arrested in the Democratic Republic of Congo over an alleged assault on one of his dancers.")

In [27]:
scores = metrics.compute(predictions=base_output8,references=targets)
scores

{'bleu': 0.12420563480222273,
 'precisions': [0.4381499894596304,
  0.17908874941070982,
  0.09682579940300837,
  0.05619688787468894],
 'brevity_penalty': 0.864057575751863,
 'length_ratio': 0.8725121280166765,
 'translation_length': 227696,
 'reference_length': 260966,
 'meteor': 0.35420455825818364,
 'rouge1': 0.4026762188767841,
 'rouge2': 0.18481796867562256,
 'rougeL': 0.3305624212280963,
 'rougeLsum': 0.3305498661301927}

In [22]:
scores45 = metrics.compute(predictions=base_output45,
                           references=targets)
scores45

{'bleu': 0.12529167445320155,
 'precisions': [0.43767581217045437,
  0.1782170041154529,
  0.0969340592861464,
  0.056633434208437666],
 'brevity_penalty': 0.8709825773070866,
 'length_ratio': 0.8786316991485481,
 'translation_length': 229293,
 'reference_length': 260966,
 'meteor': 0.35545882674212,
 'rouge1': 0.4042558166707959,
 'rouge2': 0.18471768845866826,
 'rougeL': 0.3318399755379066,
 'rougeLsum': 0.33182453398530987}

In [36]:
{'bleu': 0.13001522827171397,
 'precisions': [0.43264780948765,
  0.17764752271710285,
  0.09726789720912225,
  0.057229256856921935],
 'brevity_penalty': 0.90401138133862,
 'length_ratio': 0.9083367181931746,
 'translation_length': 237045,
 'reference_length': 260966,
 'meteor': 0.3623311725177827,
 'rouge1': 0.40646161409356885,
 'rouge2': 0.18746257090958113,
 'rougeL': 0.3322105156464427,
 'rougeLsum': 20.3334181119671774}

{'bleu': 0.1242693218959997,
 'precisions': [0.4397112710255618,
  0.17984347696769157,
  0.09751670411397062,
  0.05682880711450953],
 'brevity_penalty': 0.8588878671603885,
 'length_ratio': 0.8679674746901895,
 'translation_length': 226510,
 'reference_length': 260966,
 'meteor': 0.3532861459937222,
 'rouge1': 0.4033676736623957,
 'rouge2': 0.18549005001850066,
 'rougeL': 0.3313132654462809,
 'rougeLsum': 0.3313118204210047}

In [17]:
scores

{'bleu': 0.12215108883184916,
 'precisions': [0.42916968255609556,
  0.17216560221044527,
  0.0922525258312802,
  0.0537097051014124],
 'brevity_penalty': 0.8830712157727976,
 'length_ratio': 0.8894032172773465,
 'translation_length': 232104,
 'reference_length': 260966,
 'meteor': 0.35137424074129014,
 'rouge1': 0.398632967632497,
 'rouge2': 0.1799538886903405,
 'rougeL': 0.32609557084990337,
 'rougeLsum': 0.32622241066447516}

In [24]:
scores4

{'bleu': 0.12300035067772054,
 'precisions': [0.4394282548476454,
  0.1790089177800281,
  0.09666579620313663,
  0.056324136455435936],
 'brevity_penalty': 0.8550152182505472,
 'length_ratio': 0.8645762283209307,
 'translation_length': 225625,
 'reference_length': 260966,
 'meteor': 0.35146299353791816,
 'rouge1': 0.4017681486403819,
 'rouge2': 0.18375560820687226,
 'rougeL': 0.32963068346151203,
 'rougeLsum': 0.3296792225053877}

In [18]:
scores5

{'bleu': 0.12202486703008819,
 'precisions': [0.43851006174759005,
  0.17777018935708405,
  0.09596328825046187,
  0.05604684262301944],
 'brevity_penalty': 0.8527557381282802,
 'length_ratio': 0.8626027911681982,
 'translation_length': 225110,
 'reference_length': 260966,
 'meteor': 0.3498951876777049,
 'rouge1': 0.40069971264338244,
 'rouge2': 0.18265325890345102,
 'rougeL': 0.3290151413411355,
 'rougeLsum': 0.3290728012119742}

In [7]:
targets = [tokenizer.decode(c.labels,clean_up_tokenization_spaces=True,skip_special_tokens=True) for c in test_dataset]

Token indices sequence length is longer than the specified maximum sequence length for this model (1327 > 1024). Running this sequence through the model will result in indexing errors


In [1]:
from context_enforcement.data.common import write_to_file,read_sentences
import evaluate
metrics = evaluate.combine(["rouge"])

/home/nlplab/anaconda3/envs/development/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
import numpy as np

metrics = evaluate.combine(["bleu","meteor","rouge"])
def compute_scores(predicted,targets):
        # Rouge expects a newline after each sentence
        decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in predicted]
        decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in targets]
        
        result = metrics.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
        result2= metrics.compute(predictions=predicted, references=targets, use_stemmer=True)
        print(result)
        print(result2)
        # Extract a few results
        # result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
        
        return {k: round(v, 4) if type(v) is not list else [round(j,4) for j in v] for k, v in result.items() }

[nltk_data] Downloading package wordnet to /home/nlplab/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/nlplab/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/nlplab/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [20]:
!ls outputs/bart-context-enforcer-form1/

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
best_base_final_100.txt  best_base_final_350.txt  best_base_final_720.txt
best_base_final_200.txt  best_base_final_650.txt


In [24]:
output_path = "outputs/bart-context-enforcer-form1/best_base_final_720.txt"
# 'outputs/bart-new-crossed-context/best-22317/generated_with_contextlen_200.txt'
generated = read_sentences(output_path)

In [25]:
targets[10],generated[10]

("Queen's University Belfast is cutting 236 jobs and 290 student places due to a funding reduction.",
 "Queen's University in Belfast has announced that it is to cut 1,000 places from its undergraduate programme.")

In [26]:
compute_scores(generated,targets)

{'bleu': 0.1163854023017978, 'precisions': [0.4289432127954267, 0.1694927845773824, 0.08983130593320852, 0.05142083194951843], 'brevity_penalty': 0.8597438053136334, 'length_ratio': 0.8687185303832683, 'translation_length': 226706, 'reference_length': 260966, 'meteor': 0.34131548674181444, 'rouge1': 0.3922485320513707, 'rouge2': 0.17461115191191878, 'rougeL': 0.32054913763670956, 'rougeLsum': 0.3203804065058225}
{'bleu': 0.1163854023017978, 'precisions': [0.4289432127954267, 0.1694927845773824, 0.08983130593320852, 0.05142083194951843], 'brevity_penalty': 0.8597438053136334, 'length_ratio': 0.8687185303832683, 'translation_length': 226706, 'reference_length': 260966, 'meteor': 0.34131548674181444, 'rouge1': 0.3922485320513707, 'rouge2': 0.17461115191191878, 'rougeL': 0.32054913763670956, 'rougeLsum': 0.3203804065058225}


{'bleu': 0.1164,
 'precisions': [0.4289, 0.1695, 0.0898, 0.0514],
 'brevity_penalty': 0.8597,
 'length_ratio': 0.8687,
 'translation_length': 226706,
 'reference_length': 260966,
 'meteor': 0.3413,
 'rouge1': 0.3922,
 'rouge2': 0.1746,
 'rougeL': 0.3205,
 'rougeLsum': 0.3204}